# Test Models Tiny ImageNet-200

We are going to use the pre-trained models from [torchvision](http://pytorch.org/docs/master/torchvision/models.html) with the [Tiny ImageNet-200](https://tiny-imagenet.herokuapp.com/), a subset of ImageNet with 200 classes.

In [ ]:
%matplotlib inline

from __future__ import print_function

import json
import os

import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torch.autograd import Variable

import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torchvision.models as models

import matplotlib.pyplot as plt

Some constants for the notebook.

In [ ]:
directory = "../data/tiny-imagenet-200/"
num_classes = 200

## Loading and pre-processing

First we load and pre-process the data according to the pre-trained model [documentation](http://pytorch.org/docs/master/torchvision/models.html), applying transformations using [this example](https://github.com/pytorch/examples/blob/42e5b996718797e45c46a25c55b031e6768f8440/imagenet/main.py#L89-L113).

The training data does not really matter, we are only going to use the loader for the labels.

For the validation data we use less corrupted images, only resizing them and cropping them in the center and then appliying the rest.

In [ ]:
# modify this depending on memory constraints
batch_size = 64

# the magic normalization parameters come from the example
transform_mean = np.array([ 0.485, 0.456, 0.406 ])
transform_std = np.array([ 0.229, 0.224, 0.225 ])

train_transform = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean = transform_mean, std = transform_std),
])

val_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean = transform_mean, std = transform_std),
])

traindir = os.path.join(directory, "train")
# be careful with this set, the labels are not defined using the directory structure
valdir = os.path.join(directory, "val")

train = datasets.ImageFolder(traindir, train_transform)
val = datasets.ImageFolder(valdir, val_transform)

train_loader = torch.utils.data.DataLoader(train, batch_size=batch_size, shuffle=True)
val_loader = torch.utils.data.DataLoader(val, batch_size=batch_size, shuffle=True)

assert num_classes == len(train_loader.dataset.classes)

## Label madness

All the pre-trained models from torchvision have the same output size: 1000. Even if they are popular classes from ImageNet, it is hard to get the labels. There is a map from integers in the interval \[0, 999\] (the one hot encoded ouput) to human readable strings that can be downloaded from [here](https://gist.github.com/rcamino/ad8e7853200096f5b8cbfaa15f4a77ea).

We also build the inverse map.

In [ ]:
with open("../data/ImageNet_1000_class_ids.json", "r") as labels_file:
    labels = json.load(labels_file)
labels = dict([(int(k), v) for k, v in labels.items()])
label_ids = dict([(v, k) for k, v in labels.items()])

In [ ]:
labels.items()[:5]

In [ ]:
label_ids.items()[:5]

[WordNet](https://wordnet.princeton.edu/) is a large lexical database of English. ImageNet uses a subset of this database as labels for the images, and the Tiny ImageNet-200 uses an even smaller subset. The Tiny ImageNet-200 comes with a map between WordNet ids and WordNet definitions in the file `words.txt`.

In [ ]:
small_labels = {}
with open(os.path.join(directory, "words.txt"), "r") as dictionary_file:
    line = dictionary_file.readline()
    while line:
        label_id, label = line.strip().split("\t")
        small_labels[label_id] = label
        line = dictionary_file.readline()

In [ ]:
small_labels.items()[:5]

The train subdirectory of Tiny ImageNet-200 has a collection of subdirectories, named using to the WordNet ids to label the images that they contain. The torchvision data loader uses the names of the subdirectories as labels, but replaces them with numeric indices when iterating the batches.

Note these indices are in the range \[0, 199\] and do not match the original indices in the interval \[0, 999\].

In [ ]:
os.listdir(traindir)[:5]

In [ ]:
small_label_map = {}
for label_index, label_id in enumerate(train_loader.dataset.classes):
    label = small_labels[label_id]
    small_label_map[label_index] = label_ids[label]

In [ ]:
small_label_map.items()[:5]

Another problem is that the validation directory only has one subdirectory called `images`. The labels for every image inside this subdirectory are defined in a file called `val_annotations.txt`.

In [ ]:
val_label_map = {}
with open(os.path.join(directory, "val/val_annotations.txt"), "r") as val_label_file:
    line = val_label_file.readline()
    while line:
        file_name, label_id, _, _, _, _ = line.strip().split("\t")
        val_label_map[file_name] = label_id
        line = val_label_file.readline()

In [ ]:
val_label_map.items()[:5]

Finally we update the Tiny ImageNet-200 validation set labels using the :

In [ ]:
val_loader.dataset.imgs[:5]

In [ ]:
for i in range(len(val_loader.dataset.imgs)):
    file_path = val_loader.dataset.imgs[i][0]
    
    file_name = os.path.basename(file_path)
    label_id = val_label_map[file_name]
    label = small_labels[label_id]
    
    val_loader.dataset.imgs[i] = (file_path, label_ids[label])

In [ ]:
val_loader.dataset.imgs[:5]

## Load the model

We can chose any of the models from the [documentation](http://pytorch.org/docs/master/torchvision/models.html).

In [ ]:
model = models.densenet161(pretrained=True)
model = model.cuda()

## Plot some samples

To be sure that we did not mess up with the labels and see how the model works.

In [ ]:
num_images = 10 # modify the number of images shown

batch_inputs, batch_labels = next(iter(val_loader))
batch_inputs = Variable(batch_inputs.cuda(), volatile=True)

batch_logits = model(batch_inputs)

batch_labels = batch_labels.numpy()
batch_predictions = batch_logits.topk(5)[1].data.cpu().numpy()

cell_number = 1

plt.figure(figsize=(4, num_images * 2))

for image_number in range(num_images):
    image = np.copy(batch_inputs.data[image_number].cpu().numpy())
    image = np.transpose(image, (1, 2, 0))
    for channel in range(3):
        image[:, :, channel] = image[:, :, channel] * transform_std[channel] + transform_mean[channel]

    label = labels[batch_labels[image_number]]

    plt.subplot(num_images, 2, cell_number)

    ax = plt.imshow(image)
    plt.xticks([])
    plt.yticks([])
    
    cell_number += 1

    plt.subplot(num_images, 2, cell_number)
    plt.axis("off")
    plt.xlim(0, 1)
    plt.ylim(0, 1)
    plt.text(0, 0.85, "Label: {}".format(label))
    for prediction_number in range(5):
        plt.text(0, 0.85 - 0.15 * (prediction_number + 1), "Prediction-{:d}: {}".format(
            prediction_number + 1, labels[batch_predictions[image_number, prediction_number]]))
    
    cell_number += 1

plt.show()

## Metrics

We are going to meassure the Top-1 and Top-5 Error in the validation set.

In [ ]:
top_1 = 0.0
top_5 = 0.0
total = 0.0

log_every = 10

for batch_number, (batch_inputs, batch_labels) in enumerate(val_loader):
    batch_inputs = Variable(batch_inputs.cuda(), volatile=True)
    batch_logits = model(batch_inputs)
    
    batch_labels = batch_labels.numpy()
    batch_predictions = batch_logits.data.topk(5)[1].cpu().numpy()
            
    total += len(batch_labels)
    
    for i in range(len(batch_labels)):
        if batch_labels[i] == batch_predictions[i, 0]:
            top_1 += 1
            top_5 += 1
        else:
            for j in range(1, 5):
                if batch_labels[i] == batch_predictions[i, j]:
                    top_5 += 1
                    break
            
    if batch_number % log_every == log_every - 1:
        print("Batch {:3d} - Top-1 Error: {:.3f} Top-5 Error: {:.3f}".format(
            batch_number + 1, 100 - top_1 / total * 100, 100 - top_5 / total * 100))
        
print("Top-1 Error: {:.3f} Top-5 Error: {:.3f}".format(
    100 - top_1 / total * 100, 100 - top_5 / total * 100))